# Llama 3 8B (base) + LoRA / QLoRA (Colab용)

이 노트북은 Google Colab에서 **meta-llama/Meta-Llama-3-8B (base)**를 대상으로
- LoRA (fp16)
- QLoRA (4bit, nf4)
를 사용해 한국어 대화 요약을 학습/평가/제출용 CSV 생성까지 수행하는 코드입니다.

⚠️ 준비 사항
- Hugging Face 계정에서 `meta-llama/Meta-Llama-3-8B` 모델 액세스 허용 (이미 완료)
- HF Access Token 준비 (read 권한) (이미 완료)
- Google Drive에 데이터 저장 (`llama_summarization/data` 폴더)
- Colab GPU 런타임 (T4 / V100 / A100 등)

In [ ]:
# 1. 필수 패키지 설치

!pip install -q \
  "torch>=2.1" \
  "transformers>=4.40.0" \
  "peft>=0.10.0" \
  "accelerate>=0.30.0" \
  "bitsandbytes>=0.43.0" \
  "datasets>=2.19.0" \
  "evaluate>=0.4.0" \
  "pandas" \
  "tqdm"

In [ ]:
# 2. GPU 확인 + Google Drive 마운트 + Hugging Face 로그인

import torch, os
print("CUDA available:", torch.cuda.is_available())
try:
    from google.colab import drive
    # ⭐️ Google Drive 마운트
    drive.mount('/content/drive')
    !nvidia-smi
except Exception:
    print("Colab이 아닐 수도 있습니다. GPU 상태는 위 출력만 참고하세요.")

from huggingface_hub import login

print("\n*** Hugging Face 토큰을 입력하세요 (한 번만) ***")
# 토큰을 직접 붙여넣고 Enter
hf_token = input("Enter your Hugging Face token: ").strip()
login(token=hf_token)

In [ ]:
# 3. Google Drive에서 데이터 로딩

from pathlib import Path
import pandas as pd
import os

# ⭐️ 사용자 지정 Google Drive 경로 수정
# 파일(train.csv, dev.csv, test.csv)이 MyDrive 바로 아래에 있는 경우
ROOT_DIR = Path("/content/drive/MyDrive")

# 파일 경로를 ROOT_DIR 바로 아래로 설정
train_path = ROOT_DIR / "train.csv"
dev_path   = ROOT_DIR / "dev.csv"
test_path  = ROOT_DIR / "test.csv"

# 파일이 존재하는지 확인
assert train_path.exists(), f"파일이 없습니다: {train_path}"
assert dev_path.exists(),   f"파일이 없습니다: {dev_path}"
assert test_path.exists(),  f"파일이 없습니다: {test_path}"

train_df = pd.read_csv(train_path)
dev_df   = pd.read_csv(dev_path)
test_df  = pd.read_csv(test_path)

print(f"train/dev/test 데이터 크기: {len(train_df)}/{len(dev_df)}/{len(test_df)}")
train_df.head(), dev_df.head(), test_df.head()

In [ ]:
# 4. 프롬프트 생성 함수 + HF Dataset 변환

from datasets import Dataset

def build_prompt(dialogue: str, summary: str | None = None) -> str:
    """Llama 3 base용 Instruction-style 프롬프트."""
    system = (
        "당신은 한국어 대화 요약 비서이다.\n"
        "대화를 읽고, 한두 문장으로 핵심 내용을 간결하게 요약하라.\n"
    )
    user = f"요약: 대화의 핵심만 간결하게 한두 문장으로 정리하시오.\n\n{dialogue.strip()}\n"
    if summary is None:
        assistant = ""
    else:
        assistant = summary.strip()
    return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system}
<|start_header_id|>user<|end_header_id|>

{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{assistant}"

def make_sft_dataset(df: pd.DataFrame) -> Dataset:
    texts = [build_prompt(row["dialogue"], row["summary"]) for _, row in df.iterrows()]
    return Dataset.from_dict({"text": texts})

def make_infer_dataset(df: pd.DataFrame) -> Dataset:
    # 추론 시에는 summary를 포함하지 않습니다.
    prompts = [build_prompt(row["dialogue"], None) for _, row in df.iterrows()]
    fnames  = df["fname"].tolist() if "fname" in df.columns else list(range(len(df)))
    return Dataset.from_dict({"prompt": prompts, "fname": fnames})

train_dataset = make_sft_dataset(train_df)
dev_dataset   = make_sft_dataset(dev_df)
test_dataset  = make_infer_dataset(test_df)

print("\n--- Train Dataset Example ---")
print(train_dataset[0]["text"])

train_dataset[:2], dev_dataset[:2], test_dataset[:2]

In [ ]:
# 5. Llama 3 8B base 로드 + LoRA / QLoRA 세팅

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "meta-llama/Meta-Llama-3-8B"  # base 모델 사용
USE_4BIT = True  # True: QLoRA(4bit) - Colab T4에 필수

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16, # T4는 BF16 지원X, FP16으로 자동 전환됨
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
    )
    model = prepare_model_for_kbit_training(model)
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="auto",
    )

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

model = get_peft_model(model, lora_config)
model.config.use_cache = False  # gradient_checkpointing과 충돌 방지
model.config.pretraining_tp = 1 # 분산 학습 최적화 설정

model.print_trainable_parameters()

In [ ]:
# 6. 토크나이즈 + Trainer 세팅

from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

max_input_length = 1280 
max_new_tokens = 80 # 생성할 요약문의 최대 길이
per_device_batch_size = 1 # VRAM 16GB 한계로 1 고정 (⭐)
grad_accum_steps = 8 # 배치 크기 8의 효과를 내기 위한 경사 누적 단계 (⭐)
num_train_epochs = 3
learning_rate = 2e-4

def tokenize_sft(batch):
    return tokenizer(
        batch["text"],
        max_length=max_input_length,
        truncation=True,
        padding="max_length",
    )

train_tokenized = train_dataset.map(tokenize_sft, batched=True, remove_columns=["text"])
dev_tokenized   = dev_dataset.map(tokenize_sft,   batched=True, remove_columns=["text"])

def add_labels(batch):
    # Causal LM 학습을 위해 input_ids를 labels로 사용
    batch["labels"] = batch["input_ids"].copy()
    return batch

train_tokenized = train_tokenized.map(add_labels, batched=True)
dev_tokenized   = dev_tokenized.map(add_labels,   batched=True)

output_dir = "llama3_lora_ckpt"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_batch_size,
    per_device_eval_batch_size=per_device_batch_size,
    gradient_accumulation_steps=grad_accum_steps,
    learning_rate=learning_rate,
    num_train_epochs=num_train_epochs,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    fp16=not USE_4BIT,
    gradient_checkpointing=True, # 메모리 절약 필수 (⭐)
    load_best_model_at_end=True,
    report_to=[],
    optim="paged_adamw_8bit", # 메모리 효율적인 옵티마이저
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=dev_tokenized,
    data_collator=data_collator,
)

In [ ]:
# 7. 학습 시작

trainer.train()
# LoRA 어댑터만 저장
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
# 8. dev ROUGE 측정 (선택)

import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def generate_summary(prompt: str) -> str:
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_length,
    ).to(model.device)
    with torch.no_grad():
        # max_new_tokens만큼만 새로운 텍스트를 생성하도록 제한
        gen_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=1, # Colab 환경을 고려하여 빔 서치(Beam Search) 1로 설정하여 속도/메모리 최적화
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # 입력 프롬프트 부분을 제거하고 생성된 텍스트만 추출
    # inputs의 길이는 토큰 개수
    gen_text = tokenizer.decode(gen_ids[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
    
    # 출력에서 Llama 3 프롬프트 형식의 특수 토큰(<|eot_id|>, <|assistant|>, <|end_of_text|>) 제거
    cleaned_text = gen_text.split('<|eot_id|>')[0].strip()
    
    return cleaned_text

preds = []
refs  = []

# dev set 전체를 추론하기에는 시간이 오래 걸리므로, 샘플 100개만 사용하거나 건너뛰는 것을 고려하세요.
print("Dev Set 추론 시작...")

for i, row in dev_df.iterrows():
    if i >= 100: break # 빠른 테스트를 위해 100개만 실행
    prompt = build_prompt(row["dialogue"], None)
    pred = generate_summary(prompt)
    preds.append(pred)
    refs.append(row["summary"])

result = rouge.compute(predictions=preds, references=refs)
result = {k: round(v * 100, 2) for k, v in result.items()}
print("\n--- ROUGE 결과 (상위 100개) ---")
result

In [ ]:
# 9. test inference + 제출용 CSV 저장

summaries = []
for _, row in test_df.iterrows():
    prompt = build_prompt(row["dialogue"], None)
    fname  = row["fname"] if "fname" in row else f"test_{i}"
    summary = generate_summary(prompt)
    summaries.append({"fname": fname, "summary": summary})

pred_df = pd.DataFrame(summaries)
pred_df.to_csv("llama3_lora_test.csv", index=False)
print("\n--- 제출 파일 저장 완료 (llama3_lora_test.csv) ---")
pred_df.head()